In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Wide-Transformation").master("yarn").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/06/29 01:47:57 WARN Utils: Your hostname, Onkars-MacBook-Pro-2.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.140 instead (on interface en0)
25/06/29 01:47:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Exception in thread "main" org.apache.spark.SparkException: When running with master 'yarn' either HADOOP_CONF_DIR or YARN_CONF_DIR must be set in the environment.
	at org.apache.spark.deploy.SparkSubmitArguments.error(SparkSubmitArguments.scala:640)
	at org.apache.spark.deploy.SparkSubmitArguments.validateSubmitArguments(SparkSubmitArguments.scala:287)
	at org.apache.spark.deploy.SparkSubmitArguments.validateArguments(SparkSubmitArguments.scala:245)
	at org.apache.spark.deploy.SparkSubmitArguments.<init>(SparkSubmitArguments.scala:104)
	at org.apache.spark.deploy.SparkSubmit$$anon$2$$anon$3.<init>(SparkSubmit.scala:1101)
	at org.apache.spark.deploy.SparkSubmi

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [ ]:
customers_data = [
"customer_id,name,city,state,country,registration_date,is_active",
"0,Customer_0,Bangalore,Karnataka,India,2023-11-11,True",
"1,Customer_1,Hyderabad,Delhi,India,2023-08-26,True",
"2,Customer_2,Ahmedabad,West Bengal,India,2023-06-23,True",
"3,Customer_3,Bangalore,Tamil Nadu,India,2023-03-24,False",
"4,Customer_4,Bangalore,Gujarat,India,2023-06-06,False",
"5,Customer_5,Delhi,Maharashtra,India,2023-04-19,False"]


In [ ]:
data_rdd = spark.sparkContext.parallelize(customers_data)

In [ ]:
# Filter () -> Transformation
# first () -> Action

header = data_rdd.first()

data_rdd = data_rdd.filter(lambda row : row!=header)


In [ ]:
# first () -> Action

customer_id,name,city,state,country,registration_date,is_active


In [ ]:
def parse_data(row):
    fields = row.split(',')
    
    return (
        int(fields[0]),
        fields[1],
        fields[2],
        fields[3],
        fields[4],
        fields[5],
        fields[6]=='True'
    )

In [ ]:
parsed_rdd = data_rdd.map(parse_data)

In [ ]:
active_customers = parsed_rdd.filter(lambda row:row[6])

In [ ]:
print(parsed_rdd.getNumPartitions())

2


In [ ]:
grouped_by_city_rdd = parsed_rdd.map(lambda row: (row[2], 1)).reduceByKey(lambda x, y: x + y)

In [ ]:
grouped_by_city_rdd.collect()

[('Hyderabad', 1), ('Delhi', 1), ('Ahmedabad', 1), ('Bangalore', 3)]

In [ ]:
spark.stop()